In [1]:
print(2)

2


In [6]:
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

from tabpfn.scripts.transformer_prediction_interface import TabPFNClassifier

X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# N_ensemble_configurations controls the number of model predictions that are ensembled with feature and class rotations (See our work for details).
# When N_ensemble_configurations > #features * #classes, no further averaging is applied.

classifier = TabPFNClassifier(device='cuda:3', N_ensemble_configurations=32)

classifier.fit(X_train, y_train)
y_eval, p_eval = classifier.predict(X_test, return_winning_probability=True)

print('Accuracy', accuracy_score(y_test, y_eval))

Using style prior: True
Using cuda:3 device
Using a Transformer with 25.82 M parameters
Accuracy 0.9840425531914894


In [8]:
import torch
torch.set_grad_enabled(False)

In [9]:
# Stack tabpfn predictions with other models
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.ensemble import StackingClassifier

stack_classifier = StackingClassifier(
    estimators=[
        ('tabpfn', classifier),
        ('rf', RandomForestClassifier()),
        ('lr', LogisticRegression()),
        ('svm', SVC(probability=True))
    ],
    final_estimator=LogisticRegression()
)

stack_classifier.fit(X_train, y_train)
y_eval = stack_classifier.predict(X_test)

print('Accuracy', accuracy_score(y_test, y_eval))

Using style prior: True
Using cuda:3 device
Using a Transformer with 25.82 M parameters


/home/parietal/lgrinszt/.local/miniconda3/envs/tab_pfn/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Using style prior: True
Using cuda:3 device
Using a Transformer with 25.82 M parameters
Using style prior: True
Using cuda:3 device
Using a Transformer with 25.82 M parameters
Using style prior: True
Using cuda:3 device
Using a Transformer with 25.82 M parameters
Using style prior: True
Using cuda:3 device
Using a Transformer with 25.82 M parameters
Using style prior: True
Using cuda:3 device
Using a Transformer with 25.82 M parameters
Using style prior: True
Using cuda:3 device
Using a Transformer with 25.82 M parameters


/home/parietal/lgrinszt/.local/miniconda3/envs/tab_pfn/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/parietal/lgrinszt/.local/miniconda3/envs/tab_pfn/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https:/

Accuracy 0.9893617021276596


In [47]:
import openml
import os
openml.config.cache_directory = os.path.expanduser('/storage/store/work/lgrinszt/openml_cache')
task_id = 361071
task = openml.tasks.get_task(task_id)
X, y = task.get_X_and_y()
print(X.shape, y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Restrict X_train to 1000 samples to speed up the example
X_train, y_train = X_train[:1000], y_train[:1000]

/home/parietal/lgrinszt/.local/miniconda3/envs/tab_pfn/lib/python3.9/site-packages/arff.py:459: ResourceWarning: unclosed file <_io.TextIOWrapper name='/storage/store/work/lgrinszt/openml_cache/org/openml/www/tasks/361071/datasplits.arff' mode='r' encoding='UTF-8'>
  for row in stream:


(57580, 54) (57580,)


In [48]:
tabpfn = TabPFNClassifier(device='cuda:3', N_ensemble_configurations=32)


# try with a random forest, logistic regression, svm, and gbt
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

estimators = [
    ('tabpfn', tabpfn),
    ('rf', RandomForestClassifier()),
    ('lr', LogisticRegression()),
    ('svm', SVC(probability=True)),
    ('gbt', GradientBoostingClassifier())
]

for name, estimator in estimators:
    estimator.fit(X_train, y_train)
    y_eval = estimator.predict(X_test)
    print(name, accuracy_score(y_test, y_eval))

Using style prior: True
Using cuda:3 device
Using a Transformer with 25.82 M parameters
tabpfn 0.7206609830544153
rf 0.737290811493527


/home/parietal/lgrinszt/.local/miniconda3/envs/tab_pfn/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


lr 0.7049257972844963
svm 0.6718766445637301
gbt 0.736922429217977


In [51]:
# Try with a stacking classifier
from sklearn.ensemble import StackingClassifier

stack_classifier = StackingClassifier(
    estimators=[("tabpfn", tabpfn)],
    final_estimator=RandomForestClassifier()
)

stack_classifier.fit(X_train, y_train)
y_eval = stack_classifier.predict(X_test)
print('Stacking', accuracy_score(y_test, y_eval))


Using style prior: True
Using cuda:3 device
Using a Transformer with 25.82 M parameters
Using style prior: True
Using cuda:3 device
Using a Transformer with 25.82 M parameters
Using style prior: True
Using cuda:3 device
Using a Transformer with 25.82 M parameters
Using style prior: True
Using cuda:3 device
Using a Transformer with 25.82 M parameters
Using style prior: True
Using cuda:3 device
Using a Transformer with 25.82 M parameters
Using style prior: True
Using cuda:3 device
Using a Transformer with 25.82 M parameters
Using style prior: True
Using cuda:3 device
Using a Transformer with 25.82 M parameters
Stacking 0.6258288601199874
